In [ ]:
import requests
import numpy as np
import pandas as pd

from selenium import webdriver as wb
from bs4 import BeautifulSoup
from time import sleep
from tqdm import tqdm

In [ ]:
avia = pd.read_excel('Discrete Choice Data/Data/Parsing/aviasales_data.xlsx')

In [ ]:
avia_pairs = list(zip(avia.TC_from, avia.TC_to))

In [ ]:
len(avia_pairs)

In [ ]:
price_list = []
for pair in tqdm(avia_pairs):
    br = wb.Chrome('/Users/eugenia/Downloads/chromedriver')
    br.get('https://www.tutu.ru')
    from_port = br.find_element_by_name('city_from')
    from_port.send_keys(pair[0])
    to_port = br.find_element_by_name('city_to')
    to_port.send_keys(pair[1])
    dropdown_date = br.find_element_by_xpath('/html/body/div[3]/div[1]/div[5]/div/div[1]/div/div[2]/div[4]/div[1]/input')
    dropdown_date.click()
    br.find_element_by_xpath('/html/body/div[3]/div[1]/div[5]/div/div[1]/div/div[9]/div/table/tbody/tr[2]/td[5]/a').click()
    br.find_element_by_xpath('/html/body/div[3]/div[1]/div[5]/div/div[1]/div/div[2]/div[4]/div[2]/label/input').click()
    dropdown_class = br.find_element_by_xpath('//*[@id="wrapper"]/div[1]/div[5]/div/div[1]/div/div[7]/div/div[2]/div/div[1]')
    dropdown_class.click()
    br.find_element_by_xpath("/html/body/div[3]/div[1]/div[5]/div/div[1]/div/div[7]/div/div[2]/div/div[2]/div/ul/li[2]").click()
    sleep(0.5)
    br.find_element_by_xpath('/html/body/div[3]/div[1]/div[5]/div/div[1]/div/div[2]/div[7]/button/span/span[3]').click()
    sleep(3)
    try:
        br.get(br.current_url + 'f-changes=direct')
        sleep(0.5)
        price = br.find_element_by_xpath('/html/body/div[2]/div/div[3]/div/div[2]/div/div/div/div/div[2]/div/div/div/div/button/div[2]/div/div/span').text
        price_list.append(price)
        br.quit() 
    except: price_list.append(np.nan), br.quit() 
    

In [ ]:
avia['business_price'] = price_list
avia['business_price'] = avia.business_price.str[:-2].str.split(' ').str.join('')
avia['class_num'] = np.where(avia['class'] == 'Эконом', 0, 1)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

df_filter = avia[avia['price'] <= avia['price'].mean() + avia['price'].std() ].copy()

In [ ]:
y = df_filter.price
X = df_filter[['duration', 'class_num']]

In [ ]:
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.3)
model = LinearRegression()
model.fit(X_tr, y_tr)

In [ ]:
preds_tr = model.predict(X_tr)
preds_te = model.predict(X_te)

In [ ]:
from sklearn.metrics import mean_absolute_error

print(mean_absolute_error(y_tr, preds_tr), mean_absolute_error(y_te, preds_te))

In [ ]:
predicted_values = model.predict(avia.loc[avia.price.isnull()][['duration', 'class_num']])
filler = pd.Series(predicted_values)

In [ ]:
nan_df = avia.loc[avia.price.isna()]
notnan = avia.loc[~avia.price.isna(), 'price']
nan_df['business_price'] = list(filler)
avia['new_price'] = pd.concat([nan_df['business_price'], notnan])